# Solar Sizing for 2ha Scenario


In [1]:
#Imports 
import pvlib
from pvlib.location import Location
from pvlib import pvsystem, location, modelchain, iotools
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
import pathlib
from datetime import datetime
from dataclasses import dataclass
import sqlite3
import scipy

In [2]:
#Worst day from PlantWaterCalcs.ipynb
Q_max = 41.5008 #m3/day for 2ha
day_max = pd.to_datetime('2020-06-11').date()
max_flow_rate = (2*50*68) /(1000*3600) #m3/s
time_min = (Q_max) / (max_flow_rate* 3600) #hours


#Pump P vs Q equation from PumpCurve.ipynb --> P = a*Q^3 + b*Q^2 + c*Q + c

a = 457464099.7859799
b = -1002598.6831058152
c = 859.5461108664946
d = -0.054389800267633744

max_power = a*(max_flow_rate**3) + b*(max_flow_rate**2) + c*max_flow_rate + d #kw



In [3]:
def root_pump_curve(a,b,c,d,P,Q):
    '''function Return zero in terms of Q of pump power equation given a,b,c,d and P'''
    return (a*(Q**3) + b*(Q**2) + c*Q + d - P)

In [4]:
#Import and clean Solar data 
con = sqlite3.connect("../NetworkCode/SolarData.sqlite")
df_weather_CEBIVE_raw = pd.read_sql_query("SELECT * from cebiveSolarDailyHourly", con)
con.close()

#Convert date to datetime
df_weather_CEBIVE_raw['iso_date'] = pd.to_datetime(df_weather_CEBIVE_raw['iso_date'])
df_weather_CEBIVE_raw.index = pd.to_datetime(df_weather_CEBIVE_raw['iso_date'])

#Only data where location is CEBIVE and date is day_max
df_weather = df_weather_CEBIVE_raw[(df_weather_CEBIVE_raw['location'] == 'CEBIVE') & (df_weather_CEBIVE_raw['iso_date'].dt.date == day_max)]

#Formatting data to input into pvlib model
weather = pd.DataFrame({
    'ghi': df_weather['Cloudy_sky.ghi'], 'dhi': df_weather['Cloudy_sky.dhi'], 'dni': df_weather['Cloudy_sky.dni'],
    'temp_air':  df_weather['Temp'], 'wind_speed':  df_weather['wind_speed'], 
})

loc = location.Location(18.541, -69.990, -4.00, 32.0) #CEBIVE

In [5]:
# weather data
df_daily_irrigation =  pd.read_pickle('../NetworkCode/df_areas.pkl') #daily irrigation need in m3/day for 1h, 2ha and 3ha 
df_daily_irrigation['1ha_hrs'] = df_daily_irrigation['1 ha'] / (max_flow_rate* 3600) # convert to hrs
df_daily_irrigation['2ha_hrs'] = df_daily_irrigation['2 ha'] / (max_flow_rate* 3600) 
df_daily_irrigation['3ha_hrs'] = df_daily_irrigation['3 ha'] / (max_flow_rate* 3600) 
df_daily_irrigation.index = pd.to_datetime(df_daily_irrigation.index)


#create full weather data set
df_weather_full = df_weather_CEBIVE_raw[(df_weather_CEBIVE_raw['location'] == 'CEBIVE')]
df_weather_full.index = pd.to_datetime(df_weather_full['iso_date'])
weather_full = pd.DataFrame({
    'ghi': df_weather_full['Cloudy_sky.ghi'], 'dhi': df_weather_full['Cloudy_sky.dhi'], 'dni': df_weather_full['Cloudy_sky.dni'],
    'temp_air':  df_weather_full['Temp'], 'wind_speed':  df_weather_full['wind_speed'], 
})


## Direct drive sizing 

In [30]:
## Increase number of panels until sufficient amount to cover all historical days with variable speed operation
flag = 0 # flag == 0 --> insufficient number of panels, flag == 1 --> sufficient number of panels found 
n_panels = 1 # number of panels to use

while flag == 0: 

    #Calculate solar panel power production for all days with n panels
    loc = location.Location(18.541, -69.990, -4.00, 32.0) #CEBIVE
    solpos = loc.get_solarposition(weather_full.index)
    module_parameters = {'pdc0': 0.35, 'gamma_pdc': -0.004, 'b': 0.05} #350W pannel 
    temp_params = TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']

    array_full = pvsystem.Array(mount=pvsystem.FixedMount(20, 180),
                        module_parameters=module_parameters,
                        temperature_model_parameters=temp_params, modules_per_string=n_panels)
    system_full = pvsystem.PVSystem(arrays=[array_full], inverter_parameters={'pdc0': 5, 'eta_inv_nom':0.95, 'strings_per_inverter':1 })
    mc_full = modelchain.ModelChain(system_full, loc, spectral_model='no_loss')
    _ = mc_full.run_model(weather_full)

    df_solar_full = pd.DataFrame({'solar_power': mc_full.results.ac})

    #Calculate the flow rate [m3/s] for each hour for variable speed pump 
    df_solar_full['flow_rate'] = 0 #initialize column
    # Iterate over the index of df_solar_full directly
    for hour in df_solar_full.index:
        if df_solar_full.loc[hour, 'solar_power'] >= max_power:
            df_solar_full.loc[hour, 'flow_rate'] = max_flow_rate
        elif df_solar_full.loc[hour, 'solar_power'] == 0:
            #Handle the case when solar power is zero 
            df_solar_full.loc[hour, 'flow_rate'] = 0
        else:
            # Use root_scalar to find the root of the pump curve equation
            solution = scipy.optimize.root_scalar(lambda x: root_pump_curve(a, b, c, d, df_solar_full.loc[hour, 'solar_power'], x), x0=max_flow_rate)
            df_solar_full.loc[hour, 'flow_rate'] = solution.root

    # SAFETY: Check for negative flow rates
    negative_flow_rates = df_solar_full[df_solar_full['flow_rate'] < 0]
    if not negative_flow_rates.empty:
        raise ValueError(f"Negative flow rates found in hours: {negative_flow_rates.index.tolist()}")


    # Assuming constant flow rate over each hour, calculate the total daily pumpible water 
    df_solar_full['flow_rate_hourly']= df_solar_full['flow_rate'] * 3600
    df_daily_irrigation['2ha variable speed'] =  df_solar_full['flow_rate_hourly'].resample('D').sum()

    #Is the possible irrigaiation with variable speed operation sufficient  with n_panels? 
    day_flag = 0 # flag == 0 --> insufficient number of panels, flag == 1 --> sufficient number of panels found
    for day in df_daily_irrigation.index:
        if df_daily_irrigation['2 ha'].loc[day] > df_daily_irrigation['2ha variable speed'].loc[day]:
            print(f'{n_panels} panel(s) is not sufficient with variable speed operation, on {day} the irrigation deficit is {df_daily_irrigation["2 ha"].loc[day] - df_daily_irrigation["2ha variable speed"].loc[day]} m3')
            day_flag = 1
        
    if day_flag == 1:
        n_panels += 1
    else:
        flag = 1
        print(f'{n_panels} panel(s) is sufficient with variable speed operation')

    if n_panels > 10: #safety check
        raise ValueError('the number of PV panels required is greater than 10')         


#Takes a bit over 4 mins to run

/var/folders/_q/cqfc7f7d18ng3r1chbk8y35h0000gn/T/ipykernel_17834/4102555792.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.00014814576535805346' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_solar_full.loc[hour, 'flow_rate'] = solution.root


1 panel(s) is not sufficient with variable speed operation, on 2010-01-01 00:00:00 the irrigation deficit is 10.109559279126694 m3
1 panel(s) is not sufficient with variable speed operation, on 2010-01-02 00:00:00 the irrigation deficit is 12.666793605642315 m3
1 panel(s) is not sufficient with variable speed operation, on 2010-01-03 00:00:00 the irrigation deficit is 9.811158991301046 m3
1 panel(s) is not sufficient with variable speed operation, on 2010-01-04 00:00:00 the irrigation deficit is 8.695938059510539 m3
1 panel(s) is not sufficient with variable speed operation, on 2010-01-05 00:00:00 the irrigation deficit is 7.978158360554406 m3
1 panel(s) is not sufficient with variable speed operation, on 2010-01-06 00:00:00 the irrigation deficit is 4.9292007221235075 m3
1 panel(s) is not sufficient with variable speed operation, on 2010-01-07 00:00:00 the irrigation deficit is 6.652433845922721 m3
1 panel(s) is not sufficient with variable speed operation, on 2010-01-08 00:00:00 the 

/var/folders/_q/cqfc7f7d18ng3r1chbk8y35h0000gn/T/ipykernel_17834/4102555792.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.00032254811412650174' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_solar_full.loc[hour, 'flow_rate'] = solution.root


2 panel(s) is not sufficient with variable speed operation, on 2010-01-03 00:00:00 the irrigation deficit is 4.873947466824488 m3
2 panel(s) is not sufficient with variable speed operation, on 2010-01-08 00:00:00 the irrigation deficit is 4.160014357933827 m3
2 panel(s) is not sufficient with variable speed operation, on 2010-01-11 00:00:00 the irrigation deficit is 3.36448881113963 m3
2 panel(s) is not sufficient with variable speed operation, on 2010-01-15 00:00:00 the irrigation deficit is 5.760847406467919 m3
2 panel(s) is not sufficient with variable speed operation, on 2010-01-25 00:00:00 the irrigation deficit is 5.718362616488415 m3
2 panel(s) is not sufficient with variable speed operation, on 2010-01-26 00:00:00 the irrigation deficit is 9.366731846022885 m3
2 panel(s) is not sufficient with variable speed operation, on 2010-02-21 00:00:00 the irrigation deficit is 3.7547881878823723 m3
2 panel(s) is not sufficient with variable speed operation, on 2010-03-05 00:00:00 the irr

/var/folders/_q/cqfc7f7d18ng3r1chbk8y35h0000gn/T/ipykernel_17834/4102555792.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '7.329192682331783e-05' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_solar_full.loc[hour, 'flow_rate'] = solution.root


...

6 panel(s) is not sufficient with variable speed operation, on 2010-03-05 00:00:00 the irrigation deficit is 3.971614534770614 m3
6 panel(s) is not sufficient with variable speed operation, on 2010-03-06 00:00:00 the irrigation deficit is 0.7406403140378401 m3


/var/folders/_q/cqfc7f7d18ng3r1chbk8y35h0000gn/T/ipykernel_17834/4102555792.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.00013860108148957854' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_solar_full.loc[hour, 'flow_rate'] = solution.root


7 panel(s) is not sufficient with variable speed operation, on 2010-03-05 00:00:00 the irrigation deficit is 1.9251614855325592 m3


/var/folders/_q/cqfc7f7d18ng3r1chbk8y35h0000gn/T/ipykernel_17834/4102555792.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.00015684235027556425' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_solar_full.loc[hour, 'flow_rate'] = solution.root


8 panel(s) is sufficient with variable speed operation


## Battery system optimization 

In [8]:
# Lead acid battery characterisitics 

eta_charge = 0.95
eta_discharge = 0.95
SOC_min = 0.05 #5%
SOC_initial = 0.95 #100%
SOC_max = 0.95 #90%
deprciation = 0.001632 #depreciation per day for 8% per month
capacity = 1 #kWh

In [11]:
# Find minimum number of batteries required to cover all historical days 1 through 5 PV pannels 

for n_panels_battery in range(8, 11): #number of panels
    n_batteries = 0 #number of batteries in the string 
    flag = 0 #flag == 0 --> insufficient battery capacity found, flag == 1 --> sufficient battery capacity found

    #Calculate solar power output with n_pannels_battery
    loc = location.Location(18.541, -69.990, -4.00, 32.0) #CEBIVE
    solpos = loc.get_solarposition(weather_full.index)
    module_parameters = {'pdc0': 0.35, 'gamma_pdc': -0.004, 'b': 0.05} #350W pannel 
    temp_params = TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']

    array_full = pvsystem.Array(mount=pvsystem.FixedMount(20, 180),
                        module_parameters=module_parameters,
                        temperature_model_parameters=temp_params, modules_per_string=n_panels_battery)
    system_full = pvsystem.PVSystem(arrays=[array_full], inverter_parameters={'pdc0': 10, 'eta_inv_nom':0.95, 'strings_per_inverter':1 })
    mc_full = modelchain.ModelChain(system_full, loc, spectral_model='no_loss')
    _ = mc_full.run_model(weather_full)
    df_solar_battery = pd.DataFrame({'solar_power': mc_full.results.ac})
    

    #find minimum number of batteries needed to cover all historical days 
    while flag == 0:
        battery_capacity = capacity * n_batteries #kWh
        min_SOC = SOC_min * battery_capacity
        max_SOC = SOC_max * battery_capacity
        E_stored = [battery_capacity] #kWh, battery energy stored over time, initially full 
        day_flag = 0 
        for day, data in df_solar_battery.groupby(df_solar_battery.index.date):
            Q_day = 0 # m3 of water irrigated on day 
            Q_needed = df_daily_irrigation['2 ha'].loc[day.strftime('%Y-%m-%d')] #required irrigation in m3 
            power_day = df_solar_battery['solar_power'].loc[day.strftime('%Y-%m-%d')]
            
            for hour in data.index.hour:
                deficit = Q_needed - Q_day
                power = power_day.iloc[hour] #Power available to pump on that hour

                if deficit > 0: #Need to irrigate more water
                    if (power >= max_power) & (deficit > max_flow_rate * 3600): #Power is greater than max power so irrigate max possinle amount 
                        Q_day += max_flow_rate * 3600
                        E_storable = (power- max_power) * eta_charge #kWh
                    elif (power >= max_power) & (deficit <= max_flow_rate * 3600): #power is greater than max power but dont need to pump max amount
                        P_used = a * (deficit/3600)**3 + b * (deficit/3600)**2 + c * (deficit/3600) + d #power used to pump
                        Q_day += deficit
                        E_storable = (power- P_used) * eta_charge #kWh    
                    elif (power == 0): 
                        E_storable = 0 #no energy stored
                        Q_day += 0
                    elif (power < max_power): #power is less than max power, but still need to pump 
                        Q_possible = scipy.optimize.root_scalar(lambda x: root_pump_curve(a, b, c, d, power, x), x0=max_flow_rate)
                        E_storable = 0 #no energy stored
                        if Q_possible.root*3600 >= deficit:
                            Q_day += deficit
                        else:
                            Q_day += Q_possible.root * 3600
                    

                else: #dont need to irrigate more
                    E_storable =power*eta_charge 

                #Store energy
                if (E_storable + E_stored[-1]*(1-deprciation)) <= max_SOC:
                    E_stored.append(E_stored[-1]*(1-deprciation) + E_storable) #kWh
                else: #if more energy than battery can store, only store what battery can store
                    E_stored.append(max_SOC) #kWh
                
            
                
            if deficit > 0: #still need to irrigate more after all possible pumping hours have been exhausted
                
                #Use energy stored to pump --> Assume always pumping at max flow rate
                time_pump = (deficit / 3600)/ max_flow_rate 
                E_pump = (max_power * time_pump) / eta_discharge

                
                if (E_stored[-1] - E_pump) >= min_SOC: #if enough energy is stored to pump
                    E_stored.append(E_stored[-1]*(1-deprciation) - E_pump)
                else: #if not enough energy is stored to pump
                    #print(f'{n_batteries} batteries is not sufficient with {n_panels_battery} panels, on {day} the energy deficit is {E_pump - (E_stored[-1]- min_SOC)} kWh')
                    day_flag = 1 #flag == 1 --> n_panels_battery is not sufficient with battery operation over historical data

        if day_flag == 1: #after checking all days, if n_batteries is not sufficient with n_panels_battery
            n_batteries+=1
        else: #after checking all days, n_batteries is sufficient with n_panels_battery
            flag = 1 
            print(f'{n_batteries} Lithuim batteries is sufficient with {n_panels_battery} panels over historical data')
                
                    
        if n_batteries > 10: #safety check
            print(f'the required number of batteries is more than 10 for {n_panels_battery} panels')
            flag = 1
            
        
    #Takes 7m 55 with max 10 batteries
    #Takes 13m 4 with max 20 batteries


1 Lithuim batteries is sufficient with 8 panels over historical data
1 Lithuim batteries is sufficient with 9 panels over historical data
0 Lithuim batteries is sufficient with 10 panels over historical data


# Tank Configuration

In [13]:
def system_curve (Q): 
    "based on a given flow rate (Q) in L/min return the Head (m) of the 2ha system with values for polyfit (DripNetwork.ipynb)"
    return 0.0014811456560022407*(Q**2) + 0.014248741431001915*Q 

print(f"the needed tank height is {system_curve(max_flow_rate* 60 *1000)} m for the 2 ha system")


the needed tank height is 20.639350454831224 m for the 2 ha system


In [ ]:
def tank_pump_curve (Q):
    """based on a given flow rate (Q) in m3/s return the electrical Power (kW) of the pump for the tank configuration (H = 80.8) with variable speed op
        from PumpCurve.ipynb"""
    return 152054186.35597843*Q**3 + -172700.38396421436*Q**2 + 357.18109285749836*Q + -0.005691445084420055

tank_max_flow_rate = 0.003072606294153521 #m3/s
tank_max_power = tank_pump_curve(tank_max_flow_rate) 

def tank_system_curve_root(P, Q):
    return 152054186.35597843 * Q**3 - 172700.38396421436 * Q**2 + 357.18109285749836 * Q - 0.005691445084420055 - P
